In [ ]:
#Try to compete on https://www.kaggle.com/c/home-data-for-ml-course

# Nov 3: 17877 score
# Nov 4: 14804 score (rank: 2512)

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error


# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [13]:
X_test_full.shape

(1459, 79)

In [14]:

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [15]:


# Define model
model = RandomForestRegressor(n_estimators=500, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])


my_pipeline.fit(X_train,y_train)
preds = my_pipeline.predict(X_valid)

# Evaluate the model using cross-validation
# scores = cross_val_score(my_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

score = mean_absolute_error(y_valid, preds)

# Print the mean absolute error scores
print('MAE:', -score.mean())

MAE: -17413.575452054793


In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline

In [16]:
model2= GradientBoostingRegressor(n_estimators=500, random_state=0)
my_pipeline2 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('gbm', model2)
                             ])
my_pipeline2.fit(X_train,y_train)
preds2 =my_pipeline2.predict(X_valid)

score = mean_absolute_error(y_valid, preds2)

# Print the mean absolute error scores
print('MAE:', -score.mean())

MAE: -15965.763185108233


In [18]:
preds1_test =my_pipeline.predict(X_test)
preds2_test =my_pipeline2.predict(X_test)

preds_final = (preds1_test + preds2_test) / 2

# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_final})
output.to_csv('submission.csv', index=False)

In [20]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds2_test})
output.to_csv('submission2.csv', index=False)